In [ ]:
import os
import time
import pprint

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

from typing import Dict, Text

plt.style.use('seaborn-whitegrid')

## Data Preprocessing

#### Load dataset

In [ ]:
# Set seed for reproducibility
seed = 17
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
article_df = pd.read_csv('dataset/articles.csv')
article_top_df = pd.read_csv('dataset/articles_ranked.csv')

customer_df = pd.read_csv('dataset/customers.csv')
trans_df = pd.read_csv('dataset/transactions_train.csv')

#### Use only top-K articles

In [ ]:
TOP_K = 20000  # currently not used
N_SAMPLES = int(1e6)

In [ ]:
article_top_df.head(12)['article_id'].to_list()
percentage = article_top_df["transaction_counts"][:TOP_K].sum() / article_top_df["transaction_counts"].sum()

In [ ]:
print(f"Top {TOP_K} most sold H&M articles only account for {percentage:.2%} of the total (sold) articles")

#### Get random N samples

In [ ]:
trans_df = trans_df.sample(frac=1, random_state=seed)

In [ ]:
trans_df = trans_df.head(N_SAMPLES)

In [ ]:
len(trans_df.customer_id.unique()) / 1371980

In [ ]:
article_df.fillna(value='', inplace=True)

In [ ]:
trans_df['t_dat'] = pd.to_datetime(trans_df['t_dat']).values.astype(int) / 10**9

In [ ]:
trans_df.rename(columns={"t_dat":"timestamp"}, inplace=True)

In [ ]:
article_map = dict(article_df[['article_id', 'prod_name']].values)

In [ ]:
trans_df['prod_name'] = trans_df.article_id.map(article_map)

In [ ]:
articles = tf.data.Dataset.from_tensor_slices(dict(article_df)).map(lambda x: {
    'article_id' : tf.strings.as_string(x['article_id']),
    'prod_name' : x['prod_name'],
})

trans = tf.data.Dataset.from_tensor_slices(dict(trans_df)).map(lambda x: {
    'customer_id' : x['customer_id'],
    'article_id' : tf.strings.as_string(x['article_id']),
    'prod_name' : x['prod_name'],
    'timestamp' : x['timestamp'],
})


#### Features Preprocessing

In [ ]:
timestamps = np.concatenate(list(trans.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

In [ ]:
### USE THIS TO AVOID STRINGLOOKUP
article_ids = articles.map(lambda x: x['article_id']).batch(1_000)
unique_article_ids = np.unique(np.concatenate(list(article_ids)))

customer_ids = trans.map(lambda x: x['customer_id']).batch(1_000)
unique_customer_ids = np.unique(np.concatenate(list(customer_ids)))

## Two-tower Model

In [ ]:
class CustomerModel(tf.keras.Model):
  def __init__(self, use_timestamps):
    super().__init__()

    self._use_timestamps = use_timestamps
    
    embedding_dim = 32
    
    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_customer_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_customer_ids) + 1, embedding_dim),
    ])

    if use_timestamps:
      self.timestamp_embedding = tf.keras.Sequential([
          tf.keras.layers.Discretization(timestamp_buckets.tolist()),
          tf.keras.layers.Embedding(len(timestamp_buckets) + 1, embedding_dim),
      ])
      self.normalized_timestamp = tf.keras.layers.Normalization(
          axis=None
      )

      self.normalized_timestamp.adapt(timestamps)

  def call(self, inputs):
    print(input)
    if not self._use_timestamps:
      return self.user_embedding(inputs["customer_id"])

    return tf.concat([
        self.user_embedding(inputs["customer_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
    ], axis=1)

In [ ]:
class ArticleModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000
    
    embedding_dim = 32

    self.article_id_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_article_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_article_ids) + 1, embedding_dim)
    ])

    self.article_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.article_text_embedding = tf.keras.Sequential([
      self.article_vectorizer,
      tf.keras.layers.Embedding(max_tokens, embedding_dim, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.article_vectorizer.adapt(articles.map(lambda x: x["prod_name"]))

  def call(self, inputs):
    return tf.concat([
        self.article_id_embedding(inputs["article_id"]),
        self.article_text_embedding(inputs["prod_name"]),
    ], axis=1)

In [ ]:
class HNMModel(tfrs.models.Model):

  def __init__(self, use_timestamps):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      CustomerModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      ArticleModel(),
      tf.keras.layers.Dense(32)
    ])

    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=articles.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    query_embedding = self.query_model({
        "customer_id": features["customer_id"],
        "timestamp": features["timestamp"],
    })
    candidate_embedding = self.candidate_model({
        "article_id": features["article_id"],
        "prod_name": features["prod_name"],
    })

    return self.task(query_embedding, candidate_embedding)

## Model Training

In [ ]:
EVAL_FREQ = 1
EPOCHS = 7
INIT_EPOCH = 6  # set to latest epoch when resuming from checkpoint

In [ ]:
shuffled = trans.shuffle(trans_df.shape[0], seed=seed, reshuffle_each_iteration=False)

train = shuffled.take(800_000)
test = shuffled.skip(800_000).take(200_000)

cached_train = train.shuffle(800_000).batch(2048)
cached_test = test.batch(4096).cache()

In [ ]:
checkpoint_path = "checkpoints/test_3/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    period=EVAL_FREQ)

In [ ]:
# Load model
model = HNMModel(use_timestamps=False)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

if INIT_EPOCH > 0:
    # Load trained model weights
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)

wo_timestamp_hist = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=EVAL_FREQ,
    epochs=EPOCHS,
    initial_epoch=INIT_EPOCH,
    callbacks=[cp_callback])

In [ ]:
num_validation_runs = len(wo_timestamp_hist.history["val_factorized_top_k/top_10_categorical_accuracy"])
epochs = [(x + 1)* EVAL_FREQ for x in range(num_validation_runs)]

plt.plot(epochs, wo_timestamp_hist.history["val_factorized_top_k/top_10_categorical_accuracy"], label="w/o timesteps")
# plt.plot(epochs, w_timestamp_hist.history["val_factorized_top_k/top_10_categorical_accuracy"], label="w/ timesteps")
plt.title("Accuracy vs epoch")
plt.xlabel("epoch")
plt.ylabel("Top-100 accuracy");
plt.legend()

## Get Recommendation

In [ ]:
articles.batch(100).map(lambda x: model.candidate_model({
    'article_id': x['article_id'],
    'prod_name' : x['prod_name'],
}))

In [ ]:
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model)

identifiers = articles.batch(100).map(lambda x: x['article_id'])

candidates = articles.batch(100).map(lambda x: model.candidate_model({
        'article_id': x['article_id'],
        'prod_name' : x['prod_name'],
    }))

index.index_from_dataset(tf.data.Dataset.zip((identifiers, candidates)))

test_query = dict(trans_df[['customer_id',
                           'timestamp',
                        ]].iloc[0].map(lambda x: tf.expand_dims(x, axis=0)))

# test_query = {'customer_id' : tf.expand_dims('ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241', axis=0)}

_, titles = index(test_query, k=12)
print(f"Top 12 recommendations for user 40: {titles}")

In [ ]:
{'customer_id' : tf.expand_dims('ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38b2236865d949d4df6a', axis=0)}

In [ ]:
trans_df.loc[trans_df['customer_id'] == 'ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241']

In [ ]:
article_map[684080001]